In [2]:
import pandas as pd
import pickle
from shapely.geometry import Polygon
from shapely.geometry import MultiPoint #AMBOS HACEN LO MISMO
from scipy.spatial import ConvexHull  #AMBOS HACEN LO MISMO

In [3]:
pd.set_option('display.max_columns',None)

In [4]:
all_plays = pd.read_csv("../processed_data/weeks_frame1/frame1_process/all_players_play.csv",index_col=0)
all_plays.head()

,id,x,y,s,a,dis,o,dir,nflId,displayName,jerseyNumber,position,team,playDirection
0,2018090600:75:1,91.73,26.67,0.00,0.01,0.02,289.57,240.93,310.0,Matt Ryan,2.0,QB,away,left
1,2018090600:75:1,88.89,36.47,0.01,0.01,0.01,105.63,66.66,79848.0,Malcolm Jenkins,27.0,SS,home,left
2,2018090600:75:1,91.35,44.16,0.02,0.03,0.01,290.45,16.86,2495454.0,Julio Jones,11.0,WR,away,left
3,2018090600:75:1,86.31,22.01,0.09,0.42,0.01,70.12,168.91,2495613.0,Corey Graham,24.0,FS,home,left
4,2018090600:75:1,90.78,36.15,0.00,0.00,0.00,257.61,193.97,2533040.0,Mohamed Sanu,12.0,WR,away,left


In [5]:
defenses = pd.read_csv("../processed_data/processing_defenses/defense_players_plays_no_rz.csv",index_col=0)
defenses.head()

,id,x,y,s,a,dis,o,dir,nflId,displayName,jerseyNumber,position,team,playDirection
1,2018090600:75:1,88.89,36.47,0.01,0.01,0.01,105.63,66.66,79848.0,Malcolm Jenkins,27.0,SS,home,left
3,2018090600:75:1,86.31,22.01,0.09,0.42,0.01,70.12,168.91,2495613.0,Corey Graham,24.0,FS,home,left
5,2018090600:75:1,73.64,28.70,0.03,0.02,0.01,103.05,219.41,2534832.0,Rodney McLeod,23.0,FS,home,left
7,2018090600:75:1,86.48,31.12,0.09,0.03,0.04,95.90,33.36,2552315.0,Jordan Hicks,58.0,MLB,home,left
8,2018090600:75:1,82.67,20.53,0.03,0.07,0.01,81.14,174.57,2552689.0,Ronald Darby,21.0,CB,home,left


In [6]:
offenses = pd.read_csv("../processed_data/processing_defenses/offense_players_plays_no_rz.csv",index_col=0)
offenses.head()

,id,x,y,s,a,dis,o,dir,nflId,displayName,jerseyNumber,position,team,playDirection
0,2018090600:75:1,91.73,26.67,0.00,0.01,0.02,289.57,240.93,310.0,Matt Ryan,2.0,QB,away,left
2,2018090600:75:1,91.35,44.16,0.02,0.03,0.01,290.45,16.86,2495454.0,Julio Jones,11.0,WR,away,left
4,2018090600:75:1,90.78,36.15,0.00,0.00,0.00,257.61,193.97,2533040.0,Mohamed Sanu,12.0,WR,away,left
6,2018090600:75:1,98.25,26.63,0.01,0.01,0.00,265.88,194.51,2543583.0,Devonta Freeman,24.0,RB,away,left
10,2018090600:75:1,91.29,21.83,0.01,0.02,0.01,250.34,151.83,2555415.0,Austin Hooper,81.0,TE,away,left


In [7]:
plays = pd.read_csv("../processed_data/processing_defenses/plays.csv",index_col=0)
plays.head()

,id,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI,redZone
0,2018090600:75:1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False,False
1,2018090600:146:1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False,False
2,2018090600:168:1,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False,False
3,2018090600:190:1,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False,False
4,2018090600:256:1,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False,True


In [8]:
plays_no_rz = plays[plays["redZone"]==False]
plays_no_rz.head()

,id,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI,redZone
0,2018090600:75:1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False,False
1,2018090600:146:1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False,False
2,2018090600:168:1,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False,False
3,2018090600:190:1,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False,False
5,2018090600:320:1,(10:10) (Shotgun) N.Foles pass short left to N...,1,2,8,PHI,play_type_pass,PHI,4,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,4.0,"3 DL, 3 LB, 5 DB",0.0,0.0,10:10:00,14.0,NaN,NaN,C,4,4,-0.344096,False,False


In [ ]:
players = pd.read_csv("../processed_data/clean/players.csv",index_col=0)
players.head()

Vamos a empezar identificando cuantos jugadores hay en la zona profunda. Esta zona está 10 o 15 yardas más lejos de la línea de scrimmage.

In [12]:
transf_def = pd.DataFrame(columns = ["id"])

In [13]:
transf_def["id"] = plays_no_rz["id"]

In [32]:
def deepzonePlayers(id):
    los = plays_no_rz[plays_no_rz["id"]==id]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==id]
    playDirection = defenses[defenses["id"]==id]["playDirection"].values[0]
    deepzone = -1
    
    if playDirection == "right":
        deepzone = los + 15
    elif playDirection == "left":
        deepzone = los - 15
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]: #muy dificil que pase, pero por si acaso
            continue
        player_x = row["x"]
        if playDirection == "right" and player_x > deepzone:
            number_players+=1
        elif playDirection == "left" and player_x < deepzone:
            number_players+=1
    return number_players

In [73]:
def defensivelineZone(id): # desde la y del balón y 7 yardas para arriba y abajo
    los = plays_no_rz[plays_no_rz["id"]==id]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==id]
    playDirection = defenses[defenses["id"]==id]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==id)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_defensiveline = -1
    
    if playDirection == "right":
        x_defensiveline = los + 1.5
    elif playDirection == "left":
        x_defensiveline = los - 1.5
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            number_players+=1
            #print(row["displayName"])
        elif playDirection == "left" and player_x >= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            number_players+=1
            #print(row["displayName"])
    return number_players

In [74]:
curlzonePlayers("2018090600:3626:1")

1 Desmond Trufant
1 Robert Alford
2 Vic Beasley
1 Brian Poole
1 Damontae Kazee


5

In [75]:
defensivelineZone("2018090600:3626:1")

Vic Beasley
Deion Jones


2

In [55]:
def hookzonePlayers(id): # dentro de 23.6 y 29.7 (dar un poco de margen, 2 yardas por lado)
    los = plays_no_rz[plays_no_rz["id"]==id]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==id]
    playDirection = defenses[defenses["id"]==id]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==id)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_hook = -1
    
    if playDirection == "right":
        x_hook = los + 15
    elif playDirection == "left":
        x_hook = los - 15
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_hook and (player_y >= 21.6 and player_y <= 31.7):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and player_x > los + 1.5 :
                number_players+=1
            elif yFootball + 7 >= 31.7 and not( player_x < los + 1.5 and player_y <= 31.7 and player_y >= yFootball -7):
                number_players+=1
            elif yFootball - 7 <= 21.6 and not( player_x < los + 1.5 and player_y >= 21.6 and player_y <= yFootball +7):
                number_players+=1
        elif playDirection == "left" and player_x >= x_hook and (player_y >= 21.6 and player_y <= 31.7):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and player_x < los - 1.5 :
                number_players+=1
            elif yFootball + 7 >= 31.7 and not(player_x > los - 1.5 and player_y <= 31.7 and player_y >= yFootball - 7):
                number_players+=1
            elif yFootball - 7 <= 21.6 and not(player_x > los - 1.5 and player_y >= 21.6 and player_y <= yFootball + 7):
                number_players+=1
    return number_players

Vamos a asignar a la zona curl como la zona que está entre las hash marks y los números y la flat la zona fuera de los números

In [72]:
def curlzonePlayers(id): # está entre la zona hook y los números (12 yd hasta 21.6 yd y 31.7 yd hasta 41.3 yd)
    los = plays_no_rz[plays_no_rz["id"]==id]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==id]
    playDirection = defenses[defenses["id"]==id]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==id)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_curl = -1
    
    if playDirection == "right":
        x_curl = los + 15
    elif playDirection == "left":
        x_curl = los - 15
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_curl and ((player_y >= 12 and player_y < 21.6) or (player_y > 31.7 and player_y <= 41.3)):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and not((player_x < los +1.5 and player_y > 31.7 and player_y <= yFootball + 7) or (player_x < los + 1.5 and player_y < 21.6 and player_y >= yFootball -7)):
                number_players+=1
                #print(1,row["displayName"])
            elif yFootball + 7 > 31.7 and not(player_x < los + 1.5 and player_y > 31.7 and player_y <= yFootball + 7):
                number_players+=1
                #print(2,row["displayName"])
            elif yFootball - 7 < 21.6 and not(player_x < los + 1.5 and player_y >= yFootball - 7 and player_y < 21.6):
                number_players+=1
                #print(3,row["displayName"])
        elif playDirection == "left" and player_x >= x_curl and ((player_y >= 12 and player_y < 21.6) or (player_y > 31.7 and player_y <= 41.3)):
            if yFootball + 7 > 31.7 and yFootball - 7 < 21.6 and not((player_x > los - 1.5 and player_y > 31.7 and player_y <= yFootball + 7) or (player_x > los -1.5 and player_y < 21.6 and player_y >= yFootball - 7)):
                number_players+=1
                #print(4,row["displayName"])
            elif yFootball + 7 > 31.7 and not(player_x > los -1.5 and player_y > 31.7 and player_y <= yFootball +7):
                number_players+=1
                #print(5,row["displayName"])
            elif yFootball - 7 < 21.6 and not(player_x > los -1.5 and player_y >= yFootball - 7 and player_y < 21.6):
                number_players+=1
                #print(6,row["displayName"])
    return number_players

In [63]:
def flatzonePlayers(id): # está fuera de los números (0 yd hasta 12 yd y 41.3 yd hasta 53.3 yd)
    los = plays_no_rz[plays_no_rz["id"]==id]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==id]
    playDirection = defenses[defenses["id"]==id]["playDirection"].values[0]
    x_flat = -1
    
    if playDirection == "right":
        x_flat = los + 15
    elif playDirection == "left":
        x_flat = los - 15
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_flat and ((player_y > 0 and player_y < 12) or (player_y > 41.3 and player_y < 53.3)):
            number_players+=1
        elif playDirection == "left" and player_x >= x_flat and ((player_y > 0 and player_y < 12) or (player_y > 41.3 and player_y < 53.3)):
            number_players+=1
    return number_players

In [19]:
def numberCoverPlayers(playId):
    defenders = defenses[defenses["id"]==playId]
    
    number_players = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        else:
            number_players += 1
    
    return number_players

In [20]:
def checkNumberIsGood(dl,dp,hz,cz,fz,total):
    return dl + dp + hz +cz +fz == total

In [58]:
transf_def["defensivelinezonePlayers"] = transf_def["id"].apply(defensivelineZone)

In [59]:
transf_def["deepzonePlayers"] = transf_def["id"].apply(deepzonePlayers)

In [60]:
transf_def["hookzonePlayers"] = transf_def["id"].apply(hookzonePlayers)

In [64]:
transf_def["curlzonePlayers"] = transf_def["id"].apply(curlzonePlayers)

In [65]:
transf_def["flatzonePlayers"] = transf_def["id"].apply(flatzonePlayers)

In [66]:
transf_def["numberCoverPlayers"] = transf_def["id"].apply(numberCoverPlayers)

In [67]:
transf_def["checkNumberIsGood"] = transf_def.apply(lambda x: checkNumberIsGood(x["defensivelinezonePlayers"],x["deepzonePlayers"],x["hookzonePlayers"],x["curlzonePlayers"],x["flatzonePlayers"],x["numberCoverPlayers"]),axis=1)

In [70]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood
0,2018090600:75:1,0,1,3,2,1,7,True
1,2018090600:146:1,0,1,3,2,1,7,True
2,2018090600:168:1,0,1,2,2,2,7,True
3,2018090600:190:1,1,0,2,3,1,7,True
5,2018090600:320:1,1,0,3,3,1,8,True
...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,4,0,7,False
19113,2018123015:3932:17,0,0,1,4,2,7,True
19114,2018123015:3969:17,0,1,1,4,1,7,True
19115,2018123015:4057:17,0,0,2,3,2,7,True


In [71]:
transf_def[transf_def["checkNumberIsGood"]==False]

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood
67,2018090600:3626:1,2,0,2,5,0,8,False
68,2018090600:3678:1,1,0,4,4,0,8,False
114,2018090900:1260:1,2,0,1,7,0,8,False
119,2018090900:1551:1,2,0,1,5,2,9,False
160,2018090900:4093:1,1,0,2,4,2,8,False
...,...,...,...,...,...,...,...,...
19049,2018123015:61:17,1,0,3,4,0,7,False
19053,2018123015:349:17,1,1,3,2,1,7,False
19091,2018123015:2425:17,1,0,3,3,1,7,False
19111,2018123015:3794:17,1,0,4,3,0,7,False


In [9]:
def calculateArea(playId):
    defenders = defenses[defenses["id"]==playId]
    position_players = []
    
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
            
        position_players.append((row["x"],row["y"]))
    
    hull = ConvexHull(position_players)
    ch_area = MultiPoint(position_players).convex_hull.area
    
    return hull.volume

In [10]:
transf_def["defenseArea"] = transf_def["id"].apply(calculateArea)

In [ ]:
def calculateAreaNoLine(playId):
    defenders = defenses[defenses["id"]==playId]
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_defensiveline = -1
    
    if playDirection == "right":
        x_defensiveline = los + 1.5
    elif playDirection == "left":
        x_defensiveline = los - 1.5
    position_players = []
    
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
            
        player_x = row["x"]
        player_y = row["y"]
        
        if playDirection == "right" and player_x <= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            continue
        elif playDirection == "left" and player_x >= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            continue

        position_players.append((row["x"],row["y"]))
    
    hull = ConvexHull(position_players)
    ch_area = MultiPoint(position_players).convex_hull.area
    
    return hull.volume

In [ ]:
transf_def["defenseAreaCoverDefenders"] = transf_def["id"].apply(calculateAreaNoLine)

In [11]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,defenseArea
0,2018090600:75:1,0,1,3,2,1,194.31095
1,2018090600:146:1,0,1,3,2,1,204.17725
2,2018090600:168:1,0,1,2,2,2,277.65495
3,2018090600:190:1,1,0,2,3,1,189.16960
5,2018090600:320:1,1,0,3,3,1,187.12510
...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,4,0,169.30945
19113,2018123015:3932:17,0,0,1,4,2,327.34785
19114,2018123015:3969:17,0,1,1,4,1,293.39745
19115,2018123015:4057:17,0,0,2,3,2,223.81860


In [12]:
def calculateWidth(id):
    defenders = defenses[defenses["id"]==id]
    
    players_y = []
    
    for index,row in defenders.iterrows():
        players_y.append(row["y"])
        
    return max(players_y) - min(players_y)

In [13]:
transf_def["width"] = transf_def["id"].apply(calculateWidth)

In [14]:
def calculateHeight(id):
    defenders = defenses[defenses["id"]==id]
    
    players_x = []
    
    for index,row in defenders.iterrows():
        players_x.append(row["x"])
        
    return abs(max(players_x) - min(players_x))

In [15]:
transf_def["height"] = transf_def["id"].apply(calculateHeight)

In [16]:
def calculatePlayDirection(id):
    return defenses[defenses["id"]==id]["playDirection"].values[0]

In [17]:
transf_def["playDirection"] = transf_def["id"].apply(calculatePlayDirection)

In [18]:
def calculateNumberTEs(playId):
    offensive_players = offenses[offenses["id"]==playId]
    
    tes = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="TE")]
    
    return len(tes)

In [19]:
transf_def["numberTEs"] = transf_def["id"].apply(calculateNumberTEs)

In [20]:
def calculateNumberRBs(playId):
    offensive_players = offenses[offenses["id"]==playId]
    
    rbs = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"].isin(["HB","RB"]))]
    
    return len(rbs)

In [21]:
transf_def["numberRBs"] = transf_def["id"].apply(calculateNumberRBs)

In [22]:
def calculateNumberFBs(playId):
    offensive_players = offenses[offenses["id"]==playId]
    
    fbs = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"] == "FB")]
    
    return len(fbs)

In [23]:
transf_def["numberFBs"] = transf_def["id"].apply(calculateNumberFBs)

In [24]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,defenseArea,width,height,playDirection,numberTEs,numberRBs,numberFBs
0,2018090600:75:1,0,1,3,2,1,194.31095,22.96,15.25,left,1,1,1
1,2018090600:146:1,0,1,3,2,1,204.17725,29.22,12.57,left,1,1,0
2,2018090600:168:1,0,1,2,2,2,277.65495,33.67,15.21,left,1,2,0
3,2018090600:190:1,1,0,2,3,1,189.16960,26.42,12.91,left,1,1,0
5,2018090600:320:1,1,0,3,3,1,187.12510,27.80,13.29,right,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,4,0,169.30945,23.43,12.71,left,2,1,0
19113,2018123015:3932:17,0,0,1,4,2,327.34785,35.76,13.20,left,1,1,0
19114,2018123015:3969:17,0,1,1,4,1,293.39745,26.24,14.67,left,1,1,0
19115,2018123015:4057:17,0,0,2,3,2,223.81860,36.23,11.62,right,1,1,0


In [25]:
transf_def[transf_def["numberTEs"]==0]

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,defenseArea,width,height,playDirection,numberTEs,numberRBs,numberFBs
27,2018090600:1546:1,1,1,2,1,2,244.93380,33.71,14.16,right,0,2,0
98,2018090900:553:1,3,0,2,1,2,226.88915,35.30,12.70,right,0,1,0
321,2018090902:2955:1,0,0,2,4,1,151.24780,30.24,9.91,left,0,1,0
377,2018090903:1947:1,2,1,2,2,2,228.79630,31.37,14.89,left,0,2,0
514,2018090906:922:1,0,0,2,4,1,137.86460,20.72,10.75,left,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18937,2018123013:1399:17,1,0,3,2,1,60.83355,29.55,6.09,left,0,2,0
18941,2018123013:1723:17,1,0,3,3,0,141.40345,17.57,13.37,left,0,2,0
18942,2018123013:1795:17,0,1,3,3,0,139.40900,21.15,13.00,left,0,1,0
18953,2018123013:2329:17,1,0,4,2,0,59.21600,20.08,6.78,right,0,1,0


In [26]:
pickle_file = open('../resources/processing_defenses/transf_def.pickle','wb')
pickle.dump(transf_def,pickle_file)
pickle_file.close()

In [8]:
pickle_file_td = open('../resources/processing_defenses/transf_def.pickle','rb')
transf_def = pickle.load(pickle_file_td)
pickle_file_td.close()

Esto es auxiliar para mi

In [13]:
def calculateNumberQBs(playId):
    offensive_players = offenses[offenses["id"]==playId]
    
    qbs = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"] == "QB")]
    
    return len(qbs)

In [14]:
transf_def["numberQBs"] = transf_def["id"].apply(calculateNumberQBs)

In [15]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,defenseArea,width,height,playDirection,numberTEs,numberRBs,numberFBs,numberQBs
0,2018090600:75:1,0,1,3,2,1,194.31095,22.96,15.25,left,1,1,1,1
1,2018090600:146:1,0,1,3,2,1,204.17725,29.22,12.57,left,1,1,0,1
2,2018090600:168:1,0,1,2,2,2,277.65495,33.67,15.21,left,1,2,0,1
3,2018090600:190:1,1,0,2,3,1,189.16960,26.42,12.91,left,1,1,0,1
5,2018090600:320:1,1,0,3,3,1,187.12510,27.80,13.29,right,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,4,0,169.30945,23.43,12.71,left,2,1,0,1
19113,2018123015:3932:17,0,0,1,4,2,327.34785,35.76,13.20,left,1,1,0,1
19114,2018123015:3969:17,0,1,1,4,1,293.39745,26.24,14.67,left,1,1,0,1
19115,2018123015:4057:17,0,0,2,3,2,223.81860,36.23,11.62,right,1,1,0,1


In [21]:
transf_def[(transf_def["numberTEs"]==4)]

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,defenseArea,width,height,playDirection,numberTEs,numberRBs,numberFBs,numberQBs
14658,2018120901:3291:14,0,1,2,2,2,303.888,35.8,16.96,left,4,1,0,1


Ahora calculamos cual el es el lado fuerte

In [67]:
def calculateStrongSide(playId,numberTEs,numberFBs,numberRBs,playDirection):
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    
    strongSide = ""
    
    if numberTEs == 0:
        
    elif numberTEs == 1:
        te = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="TE")]["y"].values[0]
        if (playDirection == "left" and te > yFootball) or (playDirection == "right" and te < yFootball):
            strongSide = "right"
        elif (playDirection == "right" and te > yFootball) or (playDirection == "left" and te < yFootball):
            strongSide = "left"
    elif numberTEs == 2:
        te1 = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="TE")]["y"].values[0]
        te2 = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="TE")]["y"].values[1]
        
        if (te1 > yFootball and te2 > yFootball) or (te1 < yFootball and te2 < yFootball):
            sameSide = True
        elif (te1 > yFootball and te2 < yFootball) or (te1 < yFootball and te2 > yFootball):
            sameSide = False
        
        if (sameSide and playDirection == "left") or (sameSide and playDirection == "right"):
            strongSide = "right"
        elif (sameSide and playDirection == "right") or (sameSide and playDirection == "left"):
            strongSide = "left"
        elif not(sameSide) and numberFBs == 1:
            fb = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="FB")]["y"].values[0]
            if (playDirection == "left" and fb < yFootball) or (playDirection == "right" and fb >yFootball):
                strongSide = "left"
            elif (playDirection == "left" and fb > yFootball) or (playDirection == "right" and fb < yFootball):
                strongSide = "right"
                
        elif not(sameSide) and numberFBs == 0 and numberRBs == 1:
            rb = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"].isin(["RB","HB"]))]["y"].values[0]
            if (playDirection == "left" and rb < yFootball) or (playDirection == "right" and rb >yFootball):
                strongSide = "left"
            elif (playDirection == "left" and rb > yFootball) or (playDirection == "right" and rb < yFootball):
                strongSide = "right"
        elif not(sameSide) and numberFBs == 0 and numberRBs > 1:
            players = offensive_players[offensive_players["id"]==playId]["y"].values[0]
            number_players_fto53 = 0 #number of players from football to the line 53.3
            number_players_0tof = 0 #number of players from line 0 to football
            for index,row in players.iterrows():
                if row["position"] != "QB" and row["y"] < yFootball:
                    number_players_0tof += 1
                elif row["position"] != "QB" and row["y"] > yFootball:
                    number_players_fto53 += 1
            
            if (playDirection == "left" and number_players_fto53 > number_players_0tof) or (playDirection == "right" and number_players_fto53 < number_players_0tof) or number_players_fto53 == number_players_0tof:
                strongSide = "right"
            elif (playDirection == "left" and number_players_fto53 < number_players_0tof) or (playDirection == "right" and number_players_fto53 > number_players_0tof):
                strongSide = "left"
        elif not(sameSide) and numberFBs == 0 and numberFBs == 0:
            players = offensive_players[offensive_players["id"]==playId]["y"].values[0]
            number_players_fto53 = 0 #number of players from football to the line 53.3
            number_players_0tof = 0 #number of players from line 0 to football
            for index,row in players.iterrows():
                if row["position"] != "QB" and row["y"] < yFootball:
                    number_players_0tof += 1
                elif row["position"] != "QB" and row["y"] > yFootball:
                    number_players_fto53 += 1
            
            if (playDirection == "left" and number_players_fto53 > number_players_0tof) or (playDirection == "right" and number_players_fto53 < number_players_0tof) or number_players_fto53 == number_players_0tof:
                strongSide = "right"
            elif (playDirection == "left" and number_players_fto53 < number_players_0tof) or (playDirection == "right" and number_players_fto53 > number_players_0tof):
                strongSide = "left"
    elif numberTEs == 3:
        te1 = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="TE")]["y"].values[0]
        te2 = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="TE")]["y"].values[1]
        te3 = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="TE")]["y"].values[2]
        
        tes = [te1,te2,te3]
        
        if (te1 > yFootball and te2 > yFootball and t3 > yFootball) or (te1 < yFootball and te2 < yFootball and te3 < yFootball):
            sameSide = True
        else:jose angel
            sameSide = False
            
        if (sameSide and playDirection == "left") or (sameSide and playDirection == "right"):
            strongSide = "right"
        elif (sameSide and playDirection == "right") or (sameSide and playDirection == "left"):
            strongSide = "left"
        elif not(sameSide):
            number_players_fto53 = 0 #number of players from football to the line 53.3
            number_players_0tof = 0 #number of players from line 0 to football
            for te in tes:
                if te > yFootball:
                    number_players_fto53 += 1
                elif te < yFootball:
                    number_players_0tof +=1
            if (playDirection == "left" and number_players_fto53 > number_players_0tof) or (playDirection == "right" and number_players_fto53 < number_players_0tof):
                strongSide = "right"
            elif (playDirection == "left" and number_players_fto53 < number_players_0tof) or (playDirection == "right" and number_players_fto53 > number_players_0tof):
                strongSide = "left"
            
    elif numberTEs == 4:
        strongSide = "right"
    
    
    
        
    return strongSide

In [68]:
transf_def["strongSide"] = transf_def["id"].apply(calculateStrongSide)

In [69]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,defenseArea,points,width,height,strongSide
0,2018090600:75:1,0,1,3,2,1,180.30145,"[(88.89, 36.47), (86.48, 31.12), (85.63, 26.59...",22.96,15.25,left
1,2018090600:146:1,0,1,3,2,1,197.01345,"[(46.03, 35.49), (44.79, 30.69), (45.08, 27.2)...",29.22,12.57,left
2,2018090600:168:1,0,1,2,2,2,242.28605,"[(46.79, 44.19), (44.36, 30.92), (44.6, 28.9),...",33.67,15.21,left
3,2018090600:190:1,1,0,2,3,1,169.40415,"[(48.3, 27.21), (46.9, 22.13), (47.52, 18.83),...",26.42,12.91,left
5,2018090600:320:1,1,0,3,3,1,143.43090,"[(16.16, 38.61), (17.75, 34.45), (18.24, 30.89...",27.80,13.29,none
...,...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,4,0,96.53060,"[(62.9, 35.83), (62.13, 24.53), (65.73, 20.38)...",23.43,12.71,none
19113,2018123015:3932:17,0,0,1,4,2,184.26885,"[(41.57, 44.59), (39.9, 19.51), (42.72, 8.83),...",35.76,13.20,none
19114,2018123015:3969:17,0,1,1,4,1,196.48345,"[(45.87, 43.98), (45.94, 33.42), (43.14, 22.22...",26.24,14.67,none
19115,2018123015:4057:17,0,0,2,3,2,183.47345,"[(41.39, 45.32), (42.51, 33.78), (41.61, 24.32...",36.23,11.62,left


In [87]:
def calculateNumberPlayersSS(idPlay,strongSide,playDirection):
    yFootball = all_plays[(all_plays["id"]==idPlay)&(all_plays["displayName"]=="Football")]["y"].values[0]
    defenders = defenses[defenses["id"]==idPlay]
    
    number_players = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        
        if (playDirection == "left" and strongSide == "left" and row["y"] >= 0 and row["y"] <= yFootball) or (playDirection == "left" and strongSide == "right" and row["y"] > yFootball and row["y"] <= 53.3) or (playDirection == "right" and strongSide == "left" and row["y"] >= 53.3 and row["y"]<= yFootball) or (playDirection == "right" and strongSide == "left" and row["y"]>=0 and row["y"]<yFootball):
            number_players += 1
    
    return number_players

In [90]:
transf_def["numberPlayersStrongSide"] = transf_def.apply(lambda x: calculateNumberPlayersSS(x["id"],x["strongSide"],x["playDirection"]),axis=1)

In [91]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,defenseArea,points,width,height,strongSide,playDirection,numberPlayersStrongSide
0,2018090600:75:1,0,1,3,2,1,180.30145,"[(88.89, 36.47), (86.48, 31.12), (85.63, 26.59...",22.96,15.25,left,left,3
1,2018090600:146:1,0,1,3,2,1,197.01345,"[(46.03, 35.49), (44.79, 30.69), (45.08, 27.2)...",29.22,12.57,left,left,4
2,2018090600:168:1,0,1,2,2,2,242.28605,"[(46.79, 44.19), (44.36, 30.92), (44.6, 28.9),...",33.67,15.21,left,left,5
3,2018090600:190:1,1,0,2,3,1,169.40415,"[(48.3, 27.21), (46.9, 22.13), (47.52, 18.83),...",26.42,12.91,left,left,5
5,2018090600:320:1,1,0,3,3,1,143.43090,"[(16.16, 38.61), (17.75, 34.45), (18.24, 30.89...",27.80,13.29,none,right,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,4,0,96.53060,"[(62.9, 35.83), (62.13, 24.53), (65.73, 20.38)...",23.43,12.71,none,left,0
19113,2018123015:3932:17,0,0,1,4,2,184.26885,"[(41.57, 44.59), (39.9, 19.51), (42.72, 8.83),...",35.76,13.20,none,left,0
19114,2018123015:3969:17,0,1,1,4,1,196.48345,"[(45.87, 43.98), (45.94, 33.42), (43.14, 22.22...",26.24,14.67,none,left,0
19115,2018123015:4057:17,0,0,2,3,2,183.47345,"[(41.39, 45.32), (42.51, 33.78), (41.61, 24.32...",36.23,11.62,left,right,3


In [ ]:
def calculateNumberPlayersWS(id):
    yFootball = all_plays[(all_plays["id"]==idPlay)&(all_plays["displayName"]=="Football")]["y"].values[0]
    defenders = defenses[defenses["id"]==idPlay]
    
    number_players = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        
        if (playDirection == "left" and strongSide == "left" and row["y"] > yFootball and row["y"] <= 53.3) or (playDirection == "left" and strongSide == "right" and row["y"] >= 0 and row["y"] <= yFootball) or (playDirection == "right" and strongSide == "left" and row["y"] >= 0 and row["y"]< yFootball) or (playDirection == "right" and strongSide == "left" and row["y"]>=yFootball and row["y"]<=53.3):
            number_players += 1
    
    return number_players

In [ ]:
def calculateDiferenceWidth(playId,defenseWidth):
    offense_players = offenses[offenses["id"]==playId]
    
    players_y = []
    
    for index,row in offense_players.iterrows():
        players_y.append(row["y"])
        
    return max(players_y) - min(players_y) - defenseWidth

In [ ]:
def calculateHeightByWeightDeep(playId):
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==playId]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    deepzone = -1
    
    if playDirection == "right":
        deepzone = los + 15
    elif playDirection == "left":
        deepzone = los - 15
    
    value = 0
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]: #muy dificil que pase, pero por si acaso
            continue
        player_x = row["x"]
        if playDirection == "right" and player_x > deepzone:
            number_players+=1
            player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
            player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
            value += player_height/player_weight 
        elif playDirection == "left" and player_x < deepzone:
            number_players+=1
            player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
            player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
            value += player_height/player_weight 
            
    return value/number_players

In [ ]:
def calculateHeightByWeightHook(playId):
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==playId]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_hook = -1
    
    if playDirection == "right":
        x_hook = los + 15
    elif playDirection == "left":
        x_hook = los - 15
    
    value = 0
    number_players = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_hook and (player_y >= 21.6 and player_y <= 31.7):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and player_x > los + 1.5 :
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight 
            elif yFootball + 7 >= 31.7 and not( player_x < los + 1.5 and player_y <= 31.7 and player_y >= yFootball -7):
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight 
            elif yFootball - 7 <= 21.6 and not( player_x < los + 1.5 and player_y >= 21.6 and player_y <= yFootball +7):
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight 
        elif playDirection == "left" and player_x >= x_hook and (player_y >= 21.6 and player_y <= 31.7):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and player_x < los - 1.5 :
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight 
            elif yFootball + 7 >= 31.7 and not(player_x > los - 1.5 and player_y <= 31.7 and player_y >= yFootball - 7):
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight 
            elif yFootball - 7 <= 21.6 and not(player_x > los - 1.5 and player_y >= 21.6 and player_y <= yFootball + 7):
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight 
    
    return value/number_players

In [ ]:
def calculateHeightByWeightCurl(playId): # está entre la zona hook y los números (12 yd hasta 21.6 yd y 31.7 yd hasta 41.3 yd)
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==playId]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_curl = -1
    
    if playDirection == "right":
        x_curl = los + 15
    elif playDirection == "left":
        x_curl = los - 15
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_curl and ((player_y >= 12 and player_y < 21.6) or (player_y > 31.7 and player_y <= 41.3)):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and not((player_x < los +1.5 and player_y > 31.7 and player_y <= yFootball + 7) or (player_x < los + 1.5 and player_y < 21.6 and player_y >= yFootball -7)):
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight
            elif yFootball + 7 > 31.7 and not(player_x < los + 1.5 and player_y > 31.7 and player_y <= yFootball + 7):
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight
            elif yFootball - 7 < 21.6 and not(player_x < los + 1.5 and player_y >= yFootball - 7 and player_y < 21.6):
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight
        elif playDirection == "left" and player_x >= x_curl and ((player_y >= 12 and player_y < 21.6) or (player_y > 31.7 and player_y <= 41.3)):
            if yFootball + 7 > 31.7 and yFootball - 7 < 21.6 and not((player_x > los - 1.5 and player_y > 31.7 and player_y <= yFootball + 7) or (player_x > los -1.5 and player_y < 21.6 and player_y >= yFootball - 7)):
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight
            elif yFootball + 7 > 31.7 and not(player_x > los -1.5 and player_y > 31.7 and player_y <= yFootball +7):
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight
            elif yFootball - 7 < 21.6 and not(player_x > los -1.5 and player_y >= yFootball - 7 and player_y < 21.6):
                number_players+=1
                player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
                player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
                value += player_height/player_weight
    return value/number_players

In [ ]:
def calculateHeightByWeightFlat(playId): # está fuera de los números (0 yd hasta 12 yd y 41.3 yd hasta 53.3 yd)
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==playId]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    x_flat = -1
    
    if playDirection == "right":
        x_flat = los + 15
    elif playDirection == "left":
        x_flat = los - 15
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_flat and ((player_y > 0 and player_y < 12) or (player_y > 41.3 and player_y < 53.3)):
            number_players+=1
            player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
            player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
            value += player_height/player_weight
        elif playDirection == "left" and player_x >= x_flat and ((player_y > 0 and player_y < 12) or (player_y > 41.3 and player_y < 53.3)):
            number_players+=1
            player_height = players[players["nflId"]==row["nflId"]]["height"].values[0]
            player_weight = players[players["nflId"]==row["nflId"]]["weight"].values[0]
            value += player_height/player_weight
    return value/number_players

In [ ]:
def calculateWeightByArea(playId,area):
    defenders = defenses[defenses["id"]==playId]
    
    weight = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        weight += players[players["nflId"]==row["nflId"]]["weight"].values[0]
        
    return weight/area

In [ ]:
def calculateWeightByAreaNoLine(playId,area):
    defenders = defenses[defenses["id"]==playId]
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_defensiveline = -1
    
    if playDirection == "right":
        x_defensiveline = los + 1.5
    elif playDirection == "left":
        x_defensiveline = los - 1.5
    
    weight = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
            
        player_x = row["x"]
        player_y = row["y"]
        
        if playDirection == "right" and player_x <= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            continue
        elif playDirection == "left" and player_x >= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            continue
        weight += players[players["nflId"]==row["nflId"]]["weight"].values[0]
        
    return weight/area

Incluir numero de cbs, lbs, s y de jugadores ofensivos

In [ ]:
def calculateDensity(playId,area):
    